# Long Short-term Memory for Sentiment Classification

This notebook uses LSTM neural network on the [IMDB sentiment classification](https://keras.io/api/datasets/imdb/) task. This is a dataset for binary sentiment classification. 25,000 highly polar movie reviews are provided for training.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence


## Dataset

### Get the data
IMDB sentiment dataset is available in keras.datasets.

In [2]:
help(imdb.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.imdb:

load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).
    
    This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment
    (positive/negative). Reviews have been preprocessed, and each review is
    encoded as a list of word indexes (integers).
    For convenience, words are indexed by overall frequency in the dataset,
    so that for instance the integer "3" encodes the 3rd most frequent word in
    the data. This allows for quick filtering operations such as:
    "only consider the top 10,000 most
    common words, but eliminate the top 20 most common words".
    
    As a convention, "0" does not stand for a specific word, but instead is used
    to encode any unknown word.
    
    Args:
      path: where to cache the data (relative to `~

In [3]:
max_features = 20000
maxlen = 80
# maxlen: cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


### Data Preprocessing

Keras has already preprocessed the data

In [4]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
X_train[1]

Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


array([ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
        228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
         11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
        656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
         32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
       1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
       1355,    5,   28,    6,   52,  154,  462,   33,   89,   78,  285,
         16,  145,   95], dtype=int32)

## RNN

### Build the RNN model

In [5]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.SimpleRNN(32))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1568      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 321,601
Trainable params: 321,601
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [7]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 5s 12ms/step - loss: 0.6181 - accuracy: 0.6380 - val_loss: 0.4687 - val_accuracy: 0.7920
Epoch 2/32
313/313 [==============================] - 4s 12ms/step - loss: 0.3769 - accuracy: 0.8412 - val_loss: 0.3951 - val_accuracy: 0.8258
Epoch 3/32
313/313 [==============================] - 4s 11ms/step - loss: 0.2799 - accuracy: 0.8873 - val_loss: 0.4211 - val_accuracy: 0.8288
Epoch 4/32
313/313 [==============================] - 4s 11ms/step - loss: 0.2078 - accuracy: 0.9229 - val_loss: 0.4748 - val_accuracy: 0.7862


In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 78.53%


## RNN using the entire sequence instead of the last output

In [9]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16, input_length=maxlen))
model.add(layers.SimpleRNN(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 16)            320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 80, 32)            1568      
_________________________________________________________________
flatten (Flatten)            (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2561      
Total params: 324,129
Trainable params: 324,129
Non-trainable params: 0
_________________________________________________________________


In [10]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 6s 15ms/step - loss: 0.7053 - accuracy: 0.5047 - val_loss: 0.6841 - val_accuracy: 0.5624
Epoch 2/32
313/313 [==============================] - 4s 14ms/step - loss: 0.5614 - accuracy: 0.6943 - val_loss: 0.3885 - val_accuracy: 0.8248
Epoch 3/32
313/313 [==============================] - 5s 15ms/step - loss: 0.3429 - accuracy: 0.8504 - val_loss: 0.3446 - val_accuracy: 0.8520
Epoch 4/32
313/313 [==============================] - 4s 14ms/step - loss: 0.2782 - accuracy: 0.8817 - val_loss: 0.3439 - val_accuracy: 0.8514
Epoch 5/32
313/313 [==============================] - 5s 16ms/step - loss: 0.2396 - accuracy: 0.9017 - val_loss: 0.3557 - val_accuracy: 0.8490
Epoch 6/32
313/313 [==============================] - 5s 16ms/step - loss: 0.2132 - accuracy: 0.9171 - val_loss: 0.3553 - val_accuracy: 0.8506


In [11]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 84.10%


## LSTM

### Build the model

In [12]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 394,369
Trainable params: 394,369
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [14]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 31s 95ms/step - loss: 0.5218 - accuracy: 0.7359 - val_loss: 0.4414 - val_accuracy: 0.8216
Epoch 2/32
313/313 [==============================] - 30s 96ms/step - loss: 0.3354 - accuracy: 0.8609 - val_loss: 0.4834 - val_accuracy: 0.8274
Epoch 3/32
313/313 [==============================] - 30s 95ms/step - loss: 0.2864 - accuracy: 0.8848 - val_loss: 0.3493 - val_accuracy: 0.8480
Epoch 4/32
313/313 [==============================] - 29s 91ms/step - loss: 0.2524 - accuracy: 0.8997 - val_loss: 0.7173 - val_accuracy: 0.8122
Epoch 5/32
313/313 [==============================] - 26s 83ms/step - loss: 0.2277 - accuracy: 0.9122 - val_loss: 0.3631 - val_accuracy: 0.8352


In [15]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.64%


## Stacked LSTM

### Build the model

In [16]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         74240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 525,953
Trainable params: 525,953
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [17]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 57s 172ms/step - loss: 0.5238 - accuracy: 0.7363 - val_loss: 0.5641 - val_accuracy: 0.7500
Epoch 2/32
313/313 [==============================] - 56s 178ms/step - loss: 0.3343 - accuracy: 0.8615 - val_loss: 0.3665 - val_accuracy: 0.8444
Epoch 3/32
313/313 [==============================] - 62s 198ms/step - loss: 0.2802 - accuracy: 0.8866 - val_loss: 0.3633 - val_accuracy: 0.8432
Epoch 4/32
313/313 [==============================] - 62s 198ms/step - loss: 0.2520 - accuracy: 0.9011 - val_loss: 0.3744 - val_accuracy: 0.8422
Epoch 5/32
313/313 [==============================] - 64s 205ms/step - loss: 0.2311 - accuracy: 0.9127 - val_loss: 0.3652 - val_accuracy: 0.8458


In [18]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.99%


## Bidirectional LSTM

In [19]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               148480    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 468,737
Trainable params: 468,737
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [20]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Epoch 1/32
313/313 [==============================] - 47s 140ms/step - loss: 0.5428 - accuracy: 0.7194 - val_loss: 0.6629 - val_accuracy: 0.7516
Epoch 2/32
313/313 [==============================] - 43s 139ms/step - loss: 0.3449 - accuracy: 0.8550 - val_loss: 0.3630 - val_accuracy: 0.8402
Epoch 3/32
313/313 [==============================] - 46s 146ms/step - loss: 0.2836 - accuracy: 0.8845 - val_loss: 0.3736 - val_accuracy: 0.8422
Epoch 4/32
313/313 [==============================] - 43s 137ms/step - loss: 0.2400 - accuracy: 0.9061 - val_loss: 0.3622 - val_accuracy: 0.8384
Epoch 5/32
313/313 [==============================] - 42s 136ms/step - loss: 0.2187 - accuracy: 0.9160 - val_loss: 0.4789 - val_accuracy: 0.8326
Epoch 6/32
313/313 [==============================] - 43s 138ms/step - loss: 0.2007 - accuracy: 0.9272 - val_loss: 0.3811 - val_accuracy: 0.8476
Testing set accuracy: 83.64%
